In [1]:
import requests
import pprint
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
from config import sqlpass

In [2]:
def search_business(offset):
    api_key = "OnCmW229lglON7t4NBgK55sdvTzDfUIq-AJYVQDPnbSLwbS3rC8ky_dPBk6GETFAKBjrUfYYqBIgo5VN1G6Eu51-_gL3si3cNKJzZTxAfaiKf2YazE_1dBnkpyTuW3Yx" #  Replace this with your real API key
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

    url_params = {
                'term': "Restaurants",
                'location': "Washington+DC",
                'limit': 50,
                'offset': offset
                }
    
    response = requests.get(url, headers=headers, params=url_params)
    return response.json() 


In [3]:
if __name__ == "__main__":
    
    location = []
    
    for offset_num in np.arange(50,1000,50) :
        output_json = search_business(offset_num)
        
        for x in range(50):
            y = output_json['businesses'][x]['location']['display_address']
            y = " ".join(y)
            z = y.split( ",")
            location.append(z[0]) 
        try:
            output_json = search_business(offset_num) 
            
            if offset_num == 50:
                df_first = pd.DataFrame.from_dict(output_json['businesses'])
                        
            else:
                df2 = pd.DataFrame.from_dict(output_json['businesses'])
                df_first = df_first.append(df2)
                df_final = df_first[['name', 'rating']].copy()
                df_final["Address"] = location
                
        except AttributeError:
            print("error at ", offset_num) 
            

In [5]:
df_final = df_final.rename(columns={"name": "RestaurantName", "rating": "Rating"})
df_final.head()


,RestaurantName,Rating,Address
0,Tail Up Goat,4.5,1827 Adams Mill Rd NW Washington
1,The Haymaker Bar,4.0,1015 H St NE Washington
2,Brookland's Finest,4.0,3126 12th St NE Washington
3,Pineapple & Pearls,4.5,715 8th St SE Washington
4,DC Noodles,4.0,1412 U St NW Washington


In [18]:
rds_connection_string = f"root:{sqlpass}@127.0.0.1/restaurants_db?charset=utf8"
engine = create_engine(f'mysql://{rds_connection_string}', encoding='utf-8')

In [19]:
engine.table_names()

['google_info', 'yelp_info']

In [27]:
df_final.to_sql(name='yelp_info', con=engine, if_exists='append', index=False)

In [26]:
#This will verify we can pull the data from the table we created by combining the google and yelp data
pd.read_sql_query('select * from restaurant_info', con=engine).head(100)

,id,RestaurantName,Rating,Address
0,951,Tail Up Goat,4.5,1827 Adams Mill Rd NW Washington
1,952,The Haymaker Bar,4,1015 H St NE Washington
2,953,Brookland's Finest,4,3126 12th St NE Washington
3,954,Pineapple & Pearls,4.5,715 8th St SE Washington
4,955,DC Noodles,4,1412 U St NW Washington
5,956,Momo Yakitori,4.5,2214 Rhode Island Ave NE Washington
6,957,Homestead,4,3911 Georgia Ave NW Washington
7,958,The Dabney,4.5,122 Blagden Aly NW Washington
8,959,The Eleanor DC,4,100 Florida Ave NE Washington
9,960,Right Proper Brewing Company,4,624 T St NW Washington
